# Mission to Mars

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/beaujeffrey/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


# News

In [3]:
# URL of Red Planet Science
url_news = 'https://redplanetscience.com/'
browser.visit(url_news)

# Set up BS to find first (most recent) news title and paragraph
html = browser.html
soup = bs(html, 'html.parser')
result_news = soup.find('div', class_='list_text')

news_title = result_news.find('div', class_='content_title').text
news_paragraph = result_news.find('div', class_='article_teaser_body').text
    
print(f'Title: {news_title}')
print(f'Paragraph: {news_paragraph}')

Title: NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars
Paragraph: For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design.


# Space Image

In [4]:
# URL of Space Images - Mars
url_images = 'https://spaceimages-mars.com/'
browser.visit(url_images)

# Set up BS to find featured image
html = browser.html
soup = bs(html, 'html.parser')
result_images = soup.find('div', class_='header')
    
featured_image_url = result_images.find('img', class_='headerimage fade-in')
featured_image_url = url_images + featured_image_url['src']
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

# Mars - Earth Table

In [5]:
# URL for Mars Facts table
url_marsfacts = 'https://galaxyfacts-mars.com/'

In [6]:
# Read table
table_marsfacts = pd.read_html(url_marsfacts)
table_marsfacts

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [7]:
# Convert table to DF
marsfacts_df = table_marsfacts[0]
marsfacts_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [8]:
# Transform table to promote first row as header and set new index.
new_headers = marsfacts_df.iloc[0]
marsfacts_df = marsfacts_df[1:]
marsfacts_df.columns = new_headers
marsfacts_df = marsfacts_df.set_index('Mars - Earth Comparison')
marsfacts_df.head()

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days


In [9]:
# Convert back to html
html_table = marsfacts_df.to_html()
html_table
html_table.replace('\n', '')
marsfacts_df.to_html('table.html')

# Hemispheres

In [10]:
# URL for Hemispheres
url_hemispheres = 'https://marshemispheres.com/'
browser.visit(url_hemispheres)

# Set up BS to find routes
html = browser.html
soup = bs(html, 'html.parser')
result_hemispheres = soup.find_all('div', class_='item')

# create empty lists for gathering hemisphere names and routes to larger images
route_list = []
hemisphere_list = []

# loop through results to make lists of routes and hemisphere names
for result in result_hemispheres:
    
    route = result.find('a', class_='itemLink product-item')['href']
    route_list.append(route)
    hemisphere = result.find('h3').text
    hemisphere_list.append(hemisphere)

# create empty list to for gathering image urls
image_url_list = []

# loop through route list to make new list of image urls
for route in route_list:
    
    # attach route to end of base url
    route_url = url_hemispheres + route
    browser.visit(route_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # get route string for image
    big_image = soup.find('img', class_='wide-image')['src']
    
    # attach image route to end of base url
    image_url_list.append(url_hemispheres + str(big_image))

In [11]:
# make list of dictionaries from lists

hemi_images = [{ 'title': hemisphere_list[i],
                 'img_url': image_url_list[i] }
                for i in range(len(image_url_list)) ]
hemi_images

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]